In [5]:
import torch
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_gif
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

device = "cuda:9"
dtype = torch.float16

step = 4  # Options: [1,2,4,8]
repo = "ByteDance/AnimateDiff-Lightning"
ckpt = f"animatediff_lightning_{step}step_diffusers.safetensors"
base = "emilianJR/epiCRealism"  # Choose to your favorite base model.

adapter = MotionAdapter().to(device, dtype)
# adapter.load_state_dict(load_file(hf_hub_download(repo ,ckpt), device=device))
adapter.load_state_dict(load_file(f'/home/yiming/project/Acceleration/AnimateDiff-Lightning/AnimateDiff-Lightning/{ckpt}', device=device))
pipe = AnimateDiffPipeline.from_pretrained(base, motion_adapter=adapter, torch_dtype=dtype).to(device)
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing", beta_schedule="linear")

output = pipe(prompt="A quiet beach at dawn, the waves gently lapping at the shore and the sky painted in pastel hues.", guidance_scale=1.0, num_inference_steps=step)
export_to_gif(output.frames[0], "animation1.gif")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/home/yiming/anaconda3/envs/DiT/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
The config attributes {'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'mid_block_type': 'UNetMidBlock2DCrossAttn', 'only_cross_attention': False, 'attention_head_dim': 8, 'dual_cross_attention': False, 'class_embed_type': None, 'addition_embed_type': None, 'num_class_embeds': None, 'upcast_attention': False, 'resnet_time_scale_shift': 'default', 'resnet_skip_time_act': False, 'resnet_out_scale_factor': 1.0, 'time_embedding_type': 'positional', 'time_embedding_dim': None, 'time_embedding_act_fn': None, 'timestep_post_act': None, 'time_cond_proj_dim': None, 'conv_in_kernel': 3, 'conv_out_kernel': 3, 'projection_class_embeddings_input_dim': None, 'class_embeddings_concat': False, 'mid_b

  0%|          | 0/4 [00:00<?, ?it/s]

'animation1.gif'

In [10]:
prompts = ["Sunset over the sea.",
           "A quiet beach at dawn, the waves gently lapping at the shore and the sky painted in pastel hues.",
           "Time-lapse of a coastal landscape transitioning from sunrise to nightfall.",
           "A serene underwater scene featuring a sea turtle swimming.",
           "Yellow and black tropical fish dart through the sea.",
           "a dynamic interaction between the ocean and a large rock.",
           "The dynamic movement of tall, wispy grasses swaying in the wind.",
           "Slow pan upward of blazing oak fire in an indoor fireplace.",
           "A serene waterfall cascading down moss-covered rocks."]
for prompt in prompts:
    output = pipe(prompt=prompt, guidance_scale=1.0, num_inference_steps=step)
    export_to_gif(output.frames[0], "{}.gif".format(prompt[:30].replace(" ", "_")))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif

adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=torch.float16)
pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")

pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
# pipe.set_adapters(["lcm-lora"], [0.8])

pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

prompts = ["Sunset over the sea.",
           "A quiet beach at dawn, the waves gently lapping at the shore and the sky painted in pastel hues.",
           "Time-lapse of a coastal landscape transitioning from sunrise to nightfall.",
           "A serene underwater scene featuring a sea turtle swimming.",
           "Yellow and black tropical fish dart through the sea.",
           "a dynamic interaction between the ocean and a large rock.",
           "The dynamic movement of tall, wispy grasses swaying in the wind.",
           "Slow pan upward of blazing oak fire in an indoor fireplace.",
           "A serene waterfall cascading down moss-covered rocks."]
for prompt in prompts:
    output = pipe(prompt=prompt,
                  negative_prompt="bad quality, worse quality, low resolution",
                    num_frames=16,
                    guidance_scale=2.0,
                    num_inference_steps=6,
                    generator=torch.Generator("cpu").manual_seed(0),
                )
    export_to_gif(output.frames[0], "AnimateLCM_{}.gif".format(prompt[:30].replace(" ", "_")))
    
# output = pipe(
#     prompt="A space rocket with trails of smoke behind it launching into space from the desert, 4k, high resolution",
#     negative_prompt="bad quality, worse quality, low resolution",
#     num_frames=16,
#     guidance_scale=2.0,
#     num_inference_steps=6,
#     generator=torch.Generator("cpu").manual_seed(0),
# )
# frames = output.frames[0]
# export_to_gif(frames, "animatelcm.gif")

The config attributes {'conv_in_channels': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

The config attributes {'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'mid_block_type': 'UNetMidBlock2DCrossAttn', 'only_cross_attention': False, 'attention_head_dim': 8, 'dual_cross_attention': False, 'class_embed_type': None, 'addition_embed_type': None, 'num_class_embeds': None, 'upcast_attention': False, 'resnet_time_scale_shift': 'default', 'resnet_skip_time_act': False, 'resnet_out_scale_factor': 1.0, 'time_embedding_type': 'positional', 'time_embedding_dim': None, 'time_embedding_act_fn': None, 'timestep_post_act': None, 'time_cond_proj_dim': None, 'conv_in_kernel': 3, 'conv_out_kernel': 3, 'projection_class_embeddings_input_dim': None, 'class_embeddings_concat': False, 'mid_block_only_cross_attention': None, 'cross_attention_norm': None, 'addition_embed_type_num_heads': 64} were passed to UNetMotionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]